In [ ]:
# primero establecer el Runtime de Python 3
from google.colab import drive
drive.mount('/content/.drive')

Drive already mounted at /content/.drive; to attempt to forcibly remount, call drive.mount("/content/.drive", force_remount=True).


In [ ]:
%%shell

mkdir -p "/content/.drive/My Drive/dmeyf"
mkdir -p "/content/buckets"
ln -s "/content/.drive/My Drive/dmeyf" /content/buckets/b1

mkdir -p /content/buckets/b1/exp
mkdir -p /content/buckets/b1/datasets
mkdir -p /content/datasets

ln: failed to create symbolic link '/content/buckets/b1/dmeyf': File exists


In [ ]:
#Carga de Librerias

# limpio el ambiente de R
rm(list=ls(all.names=TRUE))
gc(full=TRUE, verbose=FALSE)

# cargo las librerias que necesito
require("data.table")
require("yaml") # Necesario si quieres trabajar con archivos de configuración

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,656471,35.1,1454465,77.7,1329246,71.0
Vcells,1219333,9.4,8388608,64.0,1975127,15.1


Loading required package: data.table

Loading required package: yaml



In [1]:
EXPERIMENTOS_A_PROMEDIAR <- c(4950, 4951, 4952, 4953, 4954, 4970, 4971, 4972, 4973, 4974)
NOMBRE_ARCHIVO_PROMEDIO <- "KA_ENSEMBLE_4950_10000_to.csv"

# Ruta
BASE_PATH <- "/content/buckets/b1/exp/"

dt_ensamble <- data.table()

for (exp in EXPERIMENTOS_A_PROMEDIAR) {
  ruta_prediccion <- paste0(BASE_PATH, "exp", exp, "/prediccion.txt")

  if (!file.exists(ruta_prediccion)) {
    cat("AVISO: Archivo de predicción no encontrado para el experimento:", exp, "\n")
    next
  }

  # Cargar el archivo de predicciones
  dt_exp <- fread(ruta_prediccion)

  # Renombrar la columna de probabilidad para identificar la semilla
  setnames(dt_exp, "prob", paste0("prob_exp_", exp))

  # Unir al ensamble
  if (nrow(dt_ensamble) == 0) {
    dt_ensamble <- dt_exp
  } else {
    dt_ensamble <- merge(
      dt_ensamble,
      dt_exp,
      by = c("numero_de_cliente", "foto_mes"),
      all = TRUE
    )
  }
}

# Calculo probabilidad media

cols_prob <- grep("^prob_exp_", colnames(dt_ensamble), value = TRUE)
dt_ensamble[, prob_final := rowMeans(.SD, na.rm = TRUE), .SDcols = cols_prob]

# Archivo final kaggle

CORTE_OPTIMO <- 11000

# Ordenar por la probabilidad promedio final
setorder(dt_ensamble, -prob_final)

# Aplicar el corte
dt_ensamble[, Predicted := 0L]
dt_ensamble[1:CORTE_OPTIMO, Predicted := 1L]

dir.create(paste0(BASE_PATH, "ENSEMBLE_KAGGLE_4950"), showWarnings = FALSE)

# Guardar el archivo final de Kaggle
ruta_kaggle_final <- paste0(BASE_PATH, "ENSEMBLE_KAGGLE_4950/", NOMBRE_ARCHIVO_PROMEDIO)

fwrite(
  dt_ensamble[, .(numero_de_cliente, Predicted)],
  file = ruta_kaggle_final,
  sep = ","
)

ERROR: Error in data.table(): could not find function "data.table"
